# Chatbot
In this notebook, we'll build a chatbot.  The chatbot interface will use gradio.  Underlying the chatbot is the Neo4j knowledge graph we built in previous labs.  The chatbot uses generative AI and langchain.  We take a natural language question, convert it to Neo4j Cypher using generative AI and run that against the database.  We take the response from the database and use generative AI again to convert that back to natural language before presenting it to the user.

Let's begin by installing the packages needed for this lab.

In [ ]:
# Used in Lab 6
%pip uninstall -y numpy
%pip install --user pandas
%pip install --user numpy==1.26.4
%pip install --user distro
%pip install --user "httpx>=0.23.0,<1"
%pip install --user langchain
%pip install --user langchain-community
%pip install --user langchain-openai
%pip install --user gradio
%pip install --user --no-cache-dir langchain-neo4j neo4j
%pip install --user openai

Now let’s restart the kernel by running the velow cell.

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(restart=True)

Click OK when the "Kernel Restarting" popup appears.

The following code cell will read the credentials you saved in the `.env` file in the previous lab and store them as variables within this notebook's environment.

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
loaded = load_dotenv()

NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")
API_ENDPOINT = os.getenv("API_ENDPOINT")
API_VERSION = os.getenv("API_VERSION")
API_KEY = os.getenv("API_KEY")
deployment_name = os.getenv("deployment_name")

if loaded:
    print("✅ Credentails loaded successfully from .env file.")
else:
    print("❌ Error: Could not load the .env file. Please ensure it exists in the correct location.")

Provide your Neo4j credentials.  We need the DB conection URL and password.  The username is neo4j by default.

## Grounding LLMs with Neo4j
Now let's create a chatbot that is grounded with Neo4j. Below is the pattern we will follow with LangChain:

![](images/langchain.png)

## Cypher Generation
We have to use a prompt template that: 
1. Clearly states what schema to use 
2. Provides principles the chatbot should follow in generating responses
3. Demonstrates few-shot examples to help the chatbot be more accurate in its query generation.

In [ ]:
CYPHER_GENERATION_TEMPLATE = """You are an expert Neo4j Cypher translator who understands the question in english and convert to Cypher strictly based on the Neo4j Schema provided and following the instructions below:
<instructions>
* Use aliases to refer the node or relationship in the generated Cypher query
* Generate Cypher query compatible ONLY for Neo4j Version 5
* Do not use EXISTS, SIZE keywords in the cypher. Use alias when using the WITH keyword
* Use only Nodes and relationships mentioned in the schema
* Always enclose the Cypher output inside 3 backticks (```)
* Always do a case-insensitive and fuzzy search for any properties related search. Eg: to search for a Company name use `toLower(c.name) contains 'neo4j'`
* Cypher is NOT SQL. So, do not mix and match the syntaxes
</instructions>

Strictly use this Schema for Cypher generation:
<schema>
{schema}
</schema>

The samples below follow the instructions and the schema mentioned above. So, please follow the same when you generate the cypher:
<samples>
Human: Which fund manager owns most shares? What is the total portfolio value?
Assistant: ```MATCH (m:Manager) -[o:OWNS]-> (c:Company) RETURN m.managerName as manager, sum(distinct o.shares) as ownedShares, sum(o.value) as portfolioValue ORDER BY ownedShares DESC LIMIT 10```

Human: Which fund manager owns most companies? How many shares?
Assistant: ```MATCH (m:Manager) -[o:OWNS]-> (c:Company) RETURN m.managerName as manager, count(distinct c) as ownedCompanies, sum(distinct o.shares) as ownedShares ORDER BY ownedCompanies DESC LIMIT 10```

Human: What are the top 10 investments for Vanguard?
Assistant: ```MATCH (m:Manager) -[o:OWNS]-> (c:Company) WHERE toLower(m.managerName) contains "vanguard" RETURN c.companyName as Investment, sum(DISTINCT o.shares) as totalShares, sum(DISTINCT o.value) as investmentValue order by investmentValue desc limit 10```

Human: What other fund managers are investing in same companies as Vanguard?
Assistant: ```MATCH (m1:Manager) -[o1:OWNS]-> (c1:Company) <-[o2:OWNS]- (m2:Manager) WHERE toLower(m1.managerName) contains "vanguard" AND elementId(m1) <> elementId(m2) RETURN m2.managerName as manager, sum(DISTINCT o2.shares) as investedShares, sum(DISTINCT o2.value) as investmentValue ORDER BY investmentValue LIMIT 10```

Human: What are the top 10 investments for rempart?
Assistant: ```MATCH (m:Manager) -[o:OWNS]-> (c:Company) WHERE toLower(m.managerName) contains "rempart" RETURN c.companyName as Investment, sum(DISTINCT o.shares) as totalShares, sum(DISTINCT o.value) as investmentValue order by investmentValue desc limit 10```

Human: What are the top investors for Apple?
Assistant: ```MATCH (m1:Manager) -[o:OWNS]-> (c1:Company) WHERE toLower(c1.companyName) contains "apple" RETURN distinct m1.managerName as manager, sum(o.value) as totalInvested ORDER BY totalInvested DESC LIMIT 10```

Human: What are the other top investments for fund managers investing in Apple?
Assistant: ```MATCH (c1:Company) <-[o1:OWNS]- (m1:Manager) -[o2:OWNS]-> (c2:Company) WHERE toLower(c1.companyName) contains "apple" AND elementId(c1) <> elementId(c2) RETURN DISTINCT c2.companyName as company, sum(o2.value) as totalInvested, sum(o2.shares) as totalShares ORDER BY totalInvested DESC LIMIT 10```

Human: What are the top investors in the last 3 months?
Assistant: ```MATCH (m:Manager) -[o:OWNS]-> (c:Company) WHERE date() > o.reportCalendarOrQuarter > o.reportCalendarOrQuarter - duration({{months:3}}) RETURN distinct m.managerName as manager, sum(o.value) as totalInvested, sum(o.shares) as totalShares ORDER BY totalInvested DESC LIMIT 10```

Human: What are top investments in last 6 months for Vanguard?
Assistant: ```MATCH (m:Manager) -[o:OWNS]-> (c:Company) WHERE toLower(m.managerName) contains "vanguard" AND date() > o.reportCalendarOrQuarter > date() - duration({{months:6}}) RETURN distinct c.companyName as company, sum(o.value) as totalInvested, sum(o.shares) as totalShares ORDER BY totalInvested DESC LIMIT 10```

Human: Who are Apple's top investors in last 3 months?
Assistant: ```MATCH (m:Manager) -[o:OWNS]-> (c:Company) WHERE toLower(c.companyName) contains "apple" AND date() > o.reportCalendarOrQuarter > date() - duration({{months:3}}) RETURN distinct m.managerName as investor, sum(o.value) as totalInvested, sum(o.shares) as totalShares ORDER BY totalInvested DESC LIMIT 10```

Human: Which fund manager under 200 million has similar investment strategy as Vanguard?
Assistant: ```MATCH (m1:Manager) -[o1:OWNS]-> (:Company) <-[o2:OWNS]- (m2:Manager) WHERE toLower(m1.managerName) CONTAINS "vanguard" AND elementId(m1) <> elementId(m2) WITH distinct m2 AS m2, sum(distinct o2.value) AS totalVal WHERE totalVal < 200000000 RETURN m2.managerName AS manager, totalVal*0.000001 AS totalVal ORDER BY totalVal DESC LIMIT 10```

Human: Who are common investors in Apple and Amazon?
Assistant: ```MATCH (c1:Company) <-[o1:OWNS]- (m:Manager) -[o2:OWNS]-> (c2:Company) WHERE toLower(c1.companyName) contains "apple" AND toLower(c2.companyName) CONTAINS "amazon" RETURN DISTINCT m.managerName LIMIT 50```

Human: What are Vanguard's top investments by shares for 2023?
Assistant: ```MATCH (m:Manager) -[o:OWNS]-> (c:Company) WHERE toLower(m.managerName) CONTAINS "vanguard" AND date({{year:2023}}) = date.truncate('year',o.reportCalendarOrQuarter) RETURN c.companyName AS investment, sum(o.value) AS totalValue ORDER BY totalValue DESC LIMIT 10```

Human: What are Vanguard's top investments by value for 2023?
Assistant: ```MATCH (m:Manager) -[o:OWNS]-> (c:Company) WHERE toLower(m.managerName) CONTAINS "vanguard" AND date({{year:2023}}) = date.truncate('year',o.reportCalendarOrQuarter) RETURN c.companyName AS investment, sum(o.shares) AS totalShares ORDER BY totalShares DESC LIMIT 10```
</samples>

Human: {question}
Assistant: 
"""

Now let’s create a LangChain prompt template.  

This template defines the parameter inputs for the prompt sent to the Cypher generation bot.  In our example, the inputs will be schema and question.  The question comes from the end user.  The LangChain GraphCypherQAChain automatically inserts the schema via a built-in method to Neo4jGraph.

In [ ]:
from langchain.prompts.prompt import PromptTemplate

CYPHER_GENERATION_PROMPT = PromptTemplate.from_template(CYPHER_GENERATION_TEMPLATE)

We need to connect to the graph via LangChain.

In [ ]:
from langchain_neo4j import Neo4jGraph

graph = Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD
)

We define our `chain` object (specifically a`GraphCypherQAChain`) using Anthropic Claude V2 LLM.

`GraphCypherQAChain` also takes a ‘Neo4jGraph’ so it can handle the chatbot process end-to-end, from taking the user question and translating to Cypher to executing the query, getting results, translating back to natural language, and returning to the user. 


In [ ]:

from langchain_neo4j import Neo4jGraph, GraphCypherQAChain
from langchain_neo4j import Neo4jGraph
from langchain.memory import ConversationBufferWindowMemory
from langchain.schema import SystemMessage, HumanMessage, AIMessage
from langchain_openai import AzureChatOpenAI 
import json

# Set up the LLM
llm = AzureChatOpenAI(
    azure_deployment=deployment_name,
    model_name=deployment_name,
    azure_endpoint=API_ENDPOINT,
    openai_api_version=API_VERSION,
    openai_api_key=API_KEY,
    temperature=0
)
chain = GraphCypherQAChain.from_llm(
    llm=llm,
    graph=graph,  # Use Neo4jGraph directly
    cypher_prompt=CYPHER_GENERATION_PROMPT,
    verbose=True,
    return_direct=True,
    allow_dangerous_requests=True,
)

def chat(que):
    r = chain.invoke(que)
    print(r)
    summary_prompt_tpl = f"""Human: 
    Fact: {json.dumps(r['result'])}

    * Summarise the above fact as if you are answering this question "{r['query']}"
    * When the fact is not empty, assume the question is valid and the answer is true
    * Do not return helpful or extra text or apologies
    * Just return summary to the user. DO NOT start with Here is a summary
    * List the results in rich text format if there are more than one results
    Assistant:
    """
    return llm.invoke(summary_prompt_tpl)

## Why Ground your LLM with Neo4j?
There are 3 primary reasons to ground your LLM with Neo4j specifically:
1. __Grounding for more complex question handling__: Multi-hop knowledge retrieval across connected data. Connections between data points are calculated before query time. 
2. __Enterprise reliability and security__: Fine-grained security so the chatbot only accesses information the user has permission to. Autonomous clustering for horizontal scaling.  Fully managed service in the cloud through Aura. 
3. __Performance__: fast queries with high concurrency for many users.

We can explore point 1 with more complex questions below.

A question requiring ~4 hops (would be joins in the relational world).  Having a knowledge graph with relationships calculated before query time allows us to answer the question quickly.

In [ ]:
r4 = chat("""What are other top investments for fund managers investing in Lowes?""")
print(f"Final answer: {r4}")

and more...

In [ ]:
r5 = chat("""Please get me common investors between Tesla and Costco""")
print(f"Final answer: {r5}")

## Grounded Chatbot
Now we will use Gradio to deploy a chat interface with our chain behind it.

The below code deploys a Gradio application.  You can access the app via a local URL. A publicly sharable URL is also provided (sharable for 3 days).

In [ ]:
import gradio as gr
from langchain.memory import ConversationBufferWindowMemory
from langchain.schema import SystemMessage, HumanMessage, AIMessage
from langchain.chat_models import AzureChatOpenAI


memory = ConversationBufferWindowMemory(
    k=5,  # Number of interactions to keep in memory.
    memory_key="chat_history",
    return_messages=True,
    output_key='output'
)

def chat_response(input_text, history):
    # Add user input as a HumanMessage to memory's chat history
    memory.chat_memory.add_message(HumanMessage(content=input_text))
    try:
        # Get the response from the chat model using the memory's chat history
        response = chat(memory.chat_memory.messages)
        # Add assistant response as an AIMessage to memory's chat history
        memory.chat_memory.add_message(AIMessage(content=response.content))
        # Return only the text content of the AIMessage
        return response.content
    except Exception as e:
        print(f"Error during chat: {e}")
        return "I'm sorry, there was an error retrieving the information you requested."

interface = gr.ChatInterface(
    fn=chat_response,
    title="Investment Chatbot",
    description="powered by Neo4j",
    theme="soft",
    chatbot=gr.Chatbot(height=500, type="messages"),
    examples=[
        "What are the top 10 investments for Vanguard?",
        "Which manager owns FAANG stocks?",
        "What are other top investments for fund managers investing in Exxon?",
        "What are Rempart's top investments by value for 2023?",
        "Who are the common investors between Tesla and Costco?",
    ],
    additional_inputs=None,
    type="messages"
)

interface.launch(share=True)

## Conclusion
In this notebook, we went through the steps of connecting a LangChain agent to a Neo4j database and using it to generate Cypher queries in response to user requests via LLMs on AzureML, thus grounding the LLM with a knowledge graph.

While we used the Azure OpenAI `gpt-4` model here, this approach can be generalized to other Azure ML LLMs.  This process can also be augmented with additional steps around the generation chain to customize the experience for specific use cases.  

The critical takeaway is the importance of Neo4j Knowledge Graph as a grounding database to: 
* Anchor your chatbot to reality as it generates responses and 
* Enable your LLM to provide answers enriched with relevant enterprise data.